In [1]:
import utils
import numpy as np

from tqdm import tqdm
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [2]:
# Note: If this cell takes fails to complete, re-run.
dji = utils.get_dji()
spy = utils.get_spy()
qqq = utils.get_qqq()

symbols = set(dji['Symbol']) | set(spy['Symbol']) | set(qqq['Symbol'])

DJI: Contains 30 tickers.
SPY: Contains 503 tickers.
QQQ: Contains 65 tickers.


In [3]:
# Note: If this cell takes fails to complete, re-run.
args = utils.load_args()
startdate, enddate = utils.get_dates(args)

market_return, riskfree_return, nonholiday_dates = utils.get_benchmark_data(args.benchmark.market, args.benchmark.riskfree, startdate, enddate)
market_data = utils.get_pdr_data(args.benchmark.market, startdate-utils.parse_period_date(args.sharpe.period), enddate, progress=False, clean=False)
stocks_data = utils.get_pdr_data(symbols, startdate-utils.parse_period_date(args.sharpe.period), enddate, progress=True, clean=True)

[*********************100%%**********************]  516 of 516 completed

11 Failed downloads:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2017-11-01 00:00:00 -> 2021-11-01 00:00:00)')
['CERN', 'FISV', 'BBBY', 'ATVI', 'BRK.B', 'CTXS']: Exception('%ticker%: No timezone found, symbol may be delisted')
['GEHC', 'CEG', 'VLTO', 'KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1509508800, endDate = 1635739200")


In [4]:
sharpe_update_dates = utils.linspace_datetime(nonholiday_dates, startdate, enddate, delta=utils.parse_period_date(args.sharpe.period))
rebalance_update_dates = utils.linspace_datetime(nonholiday_dates, startdate, enddate, delta=utils.parse_period_date(args.rebalance.period))

In [5]:
portfolio = utils.Portfolio(args.display.initial_balance)
stocks_return = utils.compute_return(stocks_data, was_annual=False, retain_symbols=True)

for dt in tqdm(nonholiday_dates, desc="nonholiday_dates"):
    if dt in rebalance_update_dates:
        # Compute sharpe.
        startdate_temp = dt - utils.parse_period_date(args.sharpe.period)
        enddate_temp = dt
        sharpe_df = utils.get_stocks_utility_from_data(stocks_data, args.benchmark.riskfree, startdate_temp, enddate_temp, args.utility)
        
        # Rebalance portfolio.
        market_sharpe = utils.get_stocks_utility_from_data(market_data, args.benchmark.riskfree, startdate_temp, enddate_temp, args.utility)
        portfolio.rebalance(sharpe_df.head(args.params.no_of_companies), min_threshold=market_sharpe[0])

        if args.display.verbose:
            print("Date:            ", dt)
            print("Market Sharpe:   ", market_sharpe[0])
            print("Portfolio $Value:", portfolio.value)
            portfolio.verbose()
            print('-'*80)

        if args.display.plot:
            portfolio.plot()
    
    # Update portfolio with today's returns.
    portfolio.update(stocks_return, dt)

nonholiday_dates: 100%|██████████| 753/753 [00:07<00:00, 104.15it/s]


In [6]:
# Final portfolio.
portfolio.verbose()

        !Utility   %Weight     $Value
Ticker                               
BX      0.201234  0.235137  50.690355
TRGP    0.192729  0.202782  43.715330
IT      0.181015  0.191721  41.330904
FTNT    0.180920  0.199522  43.012508
MS      0.168496  0.170838  36.828806


In [7]:
test_data = utils.get_pdr_data(['VTI'], startdate, enddate, progress=False, clean=False)
test_return = utils.compute_return(test_data, was_annual=False, retain_symbols=True)

print("Dates:           ", nonholiday_dates[0].date(), "to", nonholiday_dates[-1].date())
print("VTI return :     ", 100*(test_data.tail(1).values[0] - test_data.head(1).values[0]) / test_data.head(1).values[0] , "%")
print("Portfolio return:", 100*(portfolio.value-args.display.initial_balance)/args.display.initial_balance, "%")

Dates:            2018-11-01 to 2021-10-28
VTI return :      69.0644368850424 %
Portfolio return: 115.57790348217611 %


In [8]:
# import matplotlib.pyplot as plt
# import pandas as pd

# # Create a figure and axes
# fig, ax = plt.subplots()

# # Now you can add data to it when desired
# # For example, to add a line plot:
# x_values = pd.date_range(start='1/1/2020', periods=5)  # This creates a range of dates
# y_values = [1, 4, 9, 16, 25]
# ax.plot(x_values, y_values)

# # You can add more data later
# # For example, to add another line plot:
# y_values2 = [1, 8, 27, 64, 125]
# ax.plot(x_values, y_values2)

# # Rotate x-axis labels
# plt.xticks(rotation=45)

# # When you're ready to display the figure, use plt.show()
# plt.show()

In [9]:
# Couple of hings to try:
# - Double check performance when rebalancing.
# - Try Sortino's ratio instead.
# - Try also using Market Cap.